# This notebook will just attempt to correct some columns of the generated dataframes from the feature extractor functions

In [1]:
import datetime
import math
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import requests
import re
import tensorflow as tf

from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# import and load model architectures as well as decoder
from models.cueva import LSTM_FE
from models.llanes_jurado import LSTM_CNN
from utilities.preprocessors import correct_signals
from utilities.loaders import load_meta_data, save_meta_data, concur_load_data, charge_raw_data, _combine_data, load_model, load_lookup_array

from utilities.visualizers import (
    view_time_frame,
    view_wavelet_coeffs,
    analyze,
    data_split_metric_values,
    view_value_frequency,
    multi_class_heatmap,
    view_metric_values,
    view_classified_labels,
    view_label_freq,
    disp_cat_feat,
    plot_all_features,
    describe_col,
    ModelResults,
    view_all_splits_results)

from utilities.feature_extractors import (
    concur_extract_features_from_all,
    extract_features,
    extract_features_hybrid,
    extract_features_per_hour)

%load_ext autoreload
%autoreload 2

In [2]:
train_files = os.listdir('./data/Electrodermal Activity artifact correction BEnchmark (EDABE)/Train/')
train_files

['ahixac_expert1.csv',
 'akakip_expert2.csv',
 'aqamom_expert2.csv',
 'aretez_expert1.csv',
 'asifex_expert2.csv',
 'axeyoh_expert2.csv',
 'efawep_expert2.csv',
 'egemow_expert2.csv',
 'ejofeq_expert2.csv',
 'erecij_expert1.csv',
 'esirur_expert1.csv',
 'ewehov_expert2.csv',
 'exozef_expert2.csv',
 'idagah_expert2.csv',
 'ihikay_expert1.csv',
 'ihinot_expert1.csv',
 'imocac_expert2.csv',
 'iqiyat_expert2.csv',
 'obujoh_expert2.csv',
 'ohayeh_expert1.csv',
 'ohufow_expert1.csv',
 'ojotew_expert1.csv',
 'onivuk_expert1.csv',
 'opunad_expert1.csv',
 'otecab_expert2.csv',
 'otuqom_expert1.csv',
 'owegud_expert2.csv',
 'oxisux_expert1.csv',
 'tchgij_expert2.csv',
 'ufoyek_expert2.csv',
 'uqozew_expert1.csv',
 'urogif_expert1.csv',
 'uzefow_expert1.csv']

In [3]:
test_files = os.listdir('./data/Electrodermal Activity artifact correction BEnchmark (EDABE)/Test/')
test_files

['afegip_expert1.csv',
 'ajeric_expert2.csv',
 'ekamis_expert2.csv',
 'iguted_expert1.csv',
 'inefoh_expert1.csv',
 'otafeh_expert1.csv',
 'oxused_expert2.csv',
 'pqbqpr_expert2.csv',
 'uhepah_expert1.csv',
 'ukudab_expert2.csv']

In [4]:
train_subjects_names = list(set([re.sub(r".csv", "", file) for file in train_files]))
train_subjects_names

['ihinot_expert1',
 'otuqom_expert1',
 'egemow_expert2',
 'ejofeq_expert2',
 'otecab_expert2',
 'ohufow_expert1',
 'axeyoh_expert2',
 'uzefow_expert1',
 'ojotew_expert1',
 'urogif_expert1',
 'uqozew_expert1',
 'esirur_expert1',
 'ewehov_expert2',
 'aqamom_expert2',
 'onivuk_expert1',
 'ohayeh_expert1',
 'iqiyat_expert2',
 'aretez_expert1',
 'akakip_expert2',
 'idagah_expert2',
 'efawep_expert2',
 'opunad_expert1',
 'asifex_expert2',
 'obujoh_expert2',
 'ihikay_expert1',
 'imocac_expert2',
 'ufoyek_expert2',
 'erecij_expert1',
 'tchgij_expert2',
 'owegud_expert2',
 'exozef_expert2',
 'oxisux_expert1',
 'ahixac_expert1']

In [5]:
test_subjects_names = list(set([re.sub(r".csv", "", file) for file in test_files]))
test_subjects_names

['inefoh_expert1',
 'pqbqpr_expert2',
 'uhepah_expert1',
 'afegip_expert1',
 'otafeh_expert1',
 'oxused_expert2',
 'ekamis_expert2',
 'ajeric_expert2',
 'iguted_expert1',
 'ukudab_expert2']

In [6]:
all_models_results = load_meta_data('./results/all_models_results.json')
all_models_results

{'cueva_second_phase-1-5-weighted-svm': {'selected_feats': ['HOF_2',
   'HOF_5',
   'HOF_7',
   'HOF_9',
   'HOF_10',
   'HOF_12',
   'HOF_13',
   'HOF_17',
   'HOF_18',
   'HOF_19',
   'HOF_20',
   'HOF_21',
   'HOF_22',
   'HOF_26',
   'HOF_32',
   'raw_128hz_std',
   'raw_128hz_range',
   'raw_128hz_shannon_entropy',
   'raw_128hz_1d_shannon_entropy',
   'filt_128hz_min',
   'filt_128hz_1d_range',
   'filt_128hz_1d_max_abs',
   'ar_coeff_1_128hz',
   'ar_coeff_2_128hz',
   'raw_128hz_skewness',
   'raw_128hz_kurt',
   'third_16thofa_sec_max',
   'first_16thofa_sec_mean',
   'second_16thofa_sec_mean',
   'third_16thofa_sec_std',
   'first_16thofa_sec_median',
   'second_16thofa_sec_median',
   'third_16thofa_sec_range',
   'second_32thofa_sec_max',
   'first_32thofa_sec_mean',
   'second_32thofa_sec_mean',
   'first_32thofa_sec_std',
   'second_32thofa_sec_std',
   'first_32thofa_sec_median',
   'second_32thofa_sec_median',
   'first_32thofa_sec_range',
   'raw_16hz_min',
   'raw_16h

In [7]:
models = {
    # "hossain-C_1_gamma_1_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "hossain-C_1_gamma_0p1_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "hossain-C_1_gamma_0p01_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "hossain-C_1_gamma_0p001_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "hossain-C_10_gamma_1_probability_True_class_weight_None_svm": {

    # },
    # "hossain-C_10_gamma_0p1_probability_True_class_weight_None_svm": {
    # done
    # },
    # "hossain-C_10_gamma_0p01_probability_True_class_weight_None_svm": {
    #     done 
    # },
    # "hossain-C_10_gamma_0p001_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "hossain-C_100_gamma_1_probability_True_class_weight_None_svm": {

    # },
    # "hossain-C_100_gamma_0p1_probability_True_class_weight_None_svm": {

    # },
    # "hossain-C_100_gamma_0p01_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "hossain-C_100_gamma_0p001_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "hossain-C_1000_gamma_1_probability_True_class_weight_None_svm": {

    # },
    # "hossain-C_1000_gamma_0p1_probability_True_class_weight_None_svm": {

    # },
    # "hossain-C_1000_gamma_0p01_probability_True_class_weight_None_svm": {

    # },
    "hossain-C_1000_gamma_0p001_probability_True_class_weight_None_svm": {

    },

    # "taylor-C_1_gamma_1_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "taylor-C_1_gamma_0p1_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "taylor-C_1_gamma_0p01_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "taylor-C_1_gamma_0p001_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "taylor-C_10_gamma_1_probability_True_class_weight_None_svm": {

    # },
    # "taylor-C_10_gamma_0p1_probability_True_class_weight_None_svm": {
        # done
    # },
    # "taylor-C_10_gamma_0p01_probability_True_class_weight_None_svm": {
        
    # },
    # "taylor-C_10_gamma_0p001_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "taylor-C_100_gamma_1_probability_True_class_weight_None_svm": {

    # },
    # "taylor-C_100_gamma_0p1_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "taylor-C_100_gamma_0p01_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "taylor-C_100_gamma_0p001_probability_True_class_weight_None_svm": {
    #     done
    # },
    # "taylor-C_1000_gamma_1_probability_True_class_weight_None_svm": {

    # },
    "taylor-C_1000_gamma_0p1_probability_True_class_weight_None_svm": {

    },
    # "taylor-C_1000_gamma_0p01_probability_True_class_weight_None_svm": {

    # },
    # "taylor-C_1000_gamma_0p001_probability_True_class_weight_None_svm": {
    #     done
    # },

    # "hossain-C_0p01_class_weight_None_lr" : {

    # },
    # "hossain-C_0p1_class_weight_None_lr" : {

    # },
    # "hossain-C_1_class_weight_None_lr" : {

    # },
    # "hossain-C_10_class_weight_None_lr" : {

    # },
    # "hossain-C_100_class_weight_None_lr" : {

    # },
    # "taylor-C_0p01_class_weight_None_lr" : {

    # },
    # "taylor-C_0p1_class_weight_None_lr" : {

    # },
    # "taylor-C_1_class_weight_None_lr" : {

    # },
    # "taylor-C_10_class_weight_None_lr" : {

    # },
    # "taylor-C_100_class_weight_None_lr" : {

    # }
}

In [8]:
def load_miscs():
    """
    loads miscellaneous variables to be used by the model
    """

    global models

    print('loading miscellaneous...')
    # hossain_gbt_red_feats = load_lookup_array(f'./data/Artifact Detection Data/reduced_hossain_gbt_feature_set.txt')
    # taylor_rf_red_feats = load_lookup_array(f'./data/Artifact Detection Data/reduced_taylor_rf_feature_set.txt')
    hossain_svm_red_feats = load_lookup_array(f'./data/Artifact Detection Data/reduced_hossain_svm_feature_set.txt')
    taylor_svm_red_feats = load_lookup_array(f'./data/Artifact Detection Data/reduced_taylor_svm_feature_set.txt')
    hossain_lr_red_feats = load_lookup_array(f'./data/Artifact Detection Data/reduced_hossain_lr_feature_set.txt')
    taylor_lr_red_feats = load_lookup_array(f'./data/Artifact Detection Data/reduced_taylor_lr_feature_set.txt')

    # models["hossain-n_estimators_200_learning_rate_0p1_max_depth_3_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_200_learning_rate_0p01_max_depth_3_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_200_learning_rate_0p1_max_depth_5_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_200_learning_rate_0p01_max_depth_5_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_200_learning_rate_0p1_max_depth_10_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_200_learning_rate_0p01_max_depth_10_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_400_learning_rate_0p1_max_depth_3_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_400_learning_rate_0p01_max_depth_3_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_400_learning_rate_0p1_max_depth_5_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_400_learning_rate_0p01_max_depth_5_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_400_learning_rate_0p1_max_depth_10_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_400_learning_rate_0p01_max_depth_10_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_600_learning_rate_0p1_max_depth_3_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_600_learning_rate_0p01_max_depth_3_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_600_learning_rate_0p1_max_depth_5_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_600_learning_rate_0p01_max_depth_5_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_600_learning_rate_0p1_max_depth_10_gbt"]['selected_feats'] = hossain_gbt_red_feats
    # models["hossain-n_estimators_600_learning_rate_0p01_max_depth_10_gbt"]['selected_feats'] = hossain_gbt_red_feats

    # models["taylor-n_estimators_600_max_depth_50_class_weight_None_rf"]['selected_feats'] = taylor_rf_red_feats
    # models["taylor-n_estimators_400_max_depth_50_class_weight_None_rf"]['selected_feats'] = taylor_rf_red_feats
    # models["taylor-n_estimators_200_max_depth_50_class_weight_None_rf"]['selected_feats'] = taylor_rf_red_feats
    # models["taylor-n_estimators_600_max_depth_30_class_weight_None_rf"]['selected_feats'] = taylor_rf_red_feats
    # models["taylor-n_estimators_400_max_depth_30_class_weight_None_rf"]['selected_feats'] = taylor_rf_red_feats
    # models["taylor-n_estimators_200_max_depth_30_class_weight_None_rf"]['selected_feats'] = taylor_rf_red_feats
    # models["taylor-n_estimators_600_max_depth_10_class_weight_None_rf"]['selected_feats'] = taylor_rf_red_feats
    # models["taylor-n_estimators_400_max_depth_10_class_weight_None_rf"]['selected_feats'] = taylor_rf_red_feats
    # models["taylor-n_estimators_200_max_depth_10_class_weight_None_rf"]['selected_feats'] = taylor_rf_red_feats

    # done models["hossain-C_1_gamma_1_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # done models["hossain-C_1_gamma_0p1_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # done models["hossain-C_1_gamma_0p01_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # done models["hossain-C_1_gamma_0p001_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # models["hossain-C_10_gamma_1_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # models["hossain-C_10_gamma_0p1_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # done models["hossain-C_10_gamma_0p01_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # done models["hossain-C_10_gamma_0p001_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # models["hossain-C_100_gamma_1_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # models["hossain-C_100_gamma_0p1_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # done models["hossain-C_100_gamma_0p01_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # done models["hossain-C_100_gamma_0p001_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # models["hossain-C_1000_gamma_1_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # models["hossain-C_1000_gamma_0p1_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    # models["hossain-C_1000_gamma_0p01_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats
    models["hossain-C_1000_gamma_0p001_probability_True_class_weight_None_svm"]['selected_feats'] = hossain_svm_red_feats

    # done models["taylor-C_1_gamma_1_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # done models["taylor-C_1_gamma_0p1_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # done models["taylor-C_1_gamma_0p01_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # done models["taylor-C_1_gamma_0p001_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # models["taylor-C_10_gamma_1_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # done # models["taylor-C_10_gamma_0p1_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # models["taylor-C_10_gamma_0p01_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # done models["taylor-C_10_gamma_0p001_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # models["taylor-C_100_gamma_1_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # done models["taylor-C_100_gamma_0p1_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # done models["taylor-C_100_gamma_0p01_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # done models["taylor-C_100_gamma_0p001_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # models["taylor-C_1000_gamma_1_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    models["taylor-C_1000_gamma_0p1_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # models["taylor-C_1000_gamma_0p01_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats
    # done models["taylor-C_1000_gamma_0p001_probability_True_class_weight_None_svm"]['selected_feats'] = taylor_svm_red_feats

    # models["hossain-C_0p01_class_weight_None_lr"]['selected_feats'] = hossain_lr_red_feats
    # models["hossain-C_0p1_class_weight_None_lr"]['selected_feats'] = hossain_lr_red_feats
    # models["hossain-C_1_class_weight_None_lr"]['selected_feats'] = hossain_lr_red_feats
    # models["hossain-C_10_class_weight_None_lr"]['selected_feats'] = hossain_lr_red_feats
    # models["hossain-C_100_class_weight_None_lr"]['selected_feats'] = hossain_lr_red_feats

    # models["taylor-C_0p01_class_weight_None_lr"]['selected_feats'] = taylor_lr_red_feats
    # models["taylor-C_0p1_class_weight_None_lr"]['selected_feats'] = taylor_lr_red_feats
    # models["taylor-C_1_class_weight_None_lr"]['selected_feats'] = taylor_lr_red_feats
    # models["taylor-C_10_class_weight_None_lr"]['selected_feats'] = taylor_lr_red_feats
    # models["taylor-C_100_class_weight_None_lr"]['selected_feats'] = taylor_lr_red_feats

    print('miscellaneous loaded.')

def load_preprocessors():
    """
    prepares and loads the saved encoders, normalizers of
    the dataset to later transform raw user input from
    client-side
    """
    global models

    print('loading preprocessors...')

    # pre-load here scaler of hossain used during training
    # hossain_gbt_scaler = load_model('./saved/misc/hossain_gbt_scaler.pkl')
    hossain_svm_scaler = load_model('./saved/misc/hossain_svm_scaler.pkl')
    hossain_lr_scaler = load_model('./saved/misc/hossain_lr_scaler.pkl')
    
    # models["hossain-n_estimators_200_learning_rate_0p1_max_depth_3_gbt"]['scaler'] = hossain_ gbt_scaler
    # models["hossain-n_estimators_200_learning_rate_0p01_max_depth_3_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_200_learning_rate_0p1_max_depth_5_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_200_learning_rate_0p01_max_depth_5_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_200_learning_rate_0p1_max_depth_10_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_200_learning_rate_0p01_max_depth_10_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_400_learning_rate_0p1_max_depth_3_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_400_learning_rate_0p01_max_depth_3_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_400_learning_rate_0p1_max_depth_5_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_400_learning_rate_0p01_max_depth_5_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_400_learning_rate_0p1_max_depth_10_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_400_learning_rate_0p01_max_depth_10_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_600_learning_rate_0p1_max_depth_3_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_600_learning_rate_0p01_max_depth_3_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_600_learning_rate_0p1_max_depth_5_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_600_learning_rate_0p01_max_depth_5_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_600_learning_rate_0p1_max_depth_10_gbt"]['scaler'] = hossain_gbt_scaler
    # models["hossain-n_estimators_600_learning_rate_0p01_max_depth_10_gbt"]['scaler'] = hossain_gbt_scaler

    # done models["hossain-C_1_gamma_1_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # done models["hossain-C_1_gamma_0p1_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # done models["hossain-C_1_gamma_0p01_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # done models["hossain-C_1_gamma_0p001_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # models["hossain-C_10_gamma_1_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # models["hossain-C_10_gamma_0p1_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # done models["hossain-C_10_gamma_0p01_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # done models["hossain-C_10_gamma_0p001_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # models["hossain-C_100_gamma_1_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # models["hossain-C_100_gamma_0p1_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # done models["hossain-C_100_gamma_0p01_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # done models["hossain-C_100_gamma_0p001_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # models["hossain-C_1000_gamma_1_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # models["hossain-C_1000_gamma_0p1_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    # models["hossain-C_1000_gamma_0p01_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler
    models["hossain-C_1000_gamma_0p001_probability_True_class_weight_None_svm"]['scaler'] = hossain_svm_scaler

    # models["hossain-C_0p01_class_weight_None_lr"]['scaler'] = hossain_lr_scaler
    # models["hossain-C_0p1_class_weight_None_lr"]['scaler'] = hossain_lr_scaler
    # models["hossain-C_1_class_weight_None_lr"]['scaler'] = hossain_lr_scaler
    # models["hossain-C_10_class_weight_None_lr"]['scaler'] = hossain_lr_scaler
    # models["hossain-C_100_class_weight_None_lr"]['scaler'] = hossain_lr_scaler

    print('preprocessors loaded.')

def load_models():
    """
    prepares and loads sample input and custom model in
    order to use trained weights/parameters/coefficients
    """
    global models
    
    print('loading models...')
    # hossain_n_estimators_200_learning_rate_0p1_max_depth_3_gbt_clf = load_model('./saved/models/hossain_n_estimators_200_learning_rate_0p1_max_depth_3_gbt_clf.pkl')
    # hossain_n_estimators_200_learning_rate_0p01_max_depth_3_gbt_clf = load_model('./saved/models/hossain_n_estimators_200_learning_rate_0p01_max_depth_3_gbt_clf.pkl')
    # hossain_n_estimators_200_learning_rate_0p1_max_depth_5_gbt_clf = load_model('./saved/models/hossain_n_estimators_200_learning_rate_0p1_max_depth_5_gbt_clf.pkl')
    # hossain_n_estimators_200_learning_rate_0p01_max_depth_5_gbt_clf = load_model('./saved/models/hossain_n_estimators_200_learning_rate_0p01_max_depth_5_gbt_clf.pkl')
    # hossain_n_estimators_200_learning_rate_0p1_max_depth_10_gbt_clf = load_model('./saved/models/hossain_n_estimators_200_learning_rate_0p1_max_depth_10_gbt_clf.pkl')
    # hossain_n_estimators_200_learning_rate_0p01_max_depth_10_gbt_clf = load_model('./saved/models/hossain_n_estimators_200_learning_rate_0p01_max_depth_10_gbt_clf.pkl')
    # hossain_n_estimators_400_learning_rate_0p1_max_depth_3_gbt_clf = load_model('./saved/models/hossain_n_estimators_400_learning_rate_0p1_max_depth_3_gbt_clf.pkl')
    # hossain_n_estimators_400_learning_rate_0p01_max_depth_3_gbt_clf = load_model('./saved/models/hossain_n_estimators_400_learning_rate_0p01_max_depth_3_gbt_clf.pkl')
    # hossain_n_estimators_400_learning_rate_0p1_max_depth_5_gbt_clf = load_model('./saved/models/hossain_n_estimators_400_learning_rate_0p1_max_depth_5_gbt_clf.pkl')
    # hossain_n_estimators_400_learning_rate_0p01_max_depth_5_gbt_clf = load_model('./saved/models/hossain_n_estimators_400_learning_rate_0p01_max_depth_5_gbt_clf.pkl')
    # hossain_n_estimators_400_learning_rate_0p1_max_depth_10_gbt_clf = load_model('./saved/models/hossain_n_estimators_400_learning_rate_0p1_max_depth_10_gbt_clf.pkl')
    # hossain_n_estimators_400_learning_rate_0p01_max_depth_10_gbt_clf = load_model('./saved/models/hossain_n_estimators_400_learning_rate_0p01_max_depth_10_gbt_clf.pkl')
    # hossain_n_estimators_600_learning_rate_0p1_max_depth_3_gbt_clf = load_model('./saved/models/hossain_n_estimators_600_learning_rate_0p1_max_depth_3_gbt_clf.pkl')
    # hossain_n_estimators_600_learning_rate_0p01_max_depth_3_gbt_clf = load_model('./saved/models/hossain_n_estimators_600_learning_rate_0p01_max_depth_3_gbt_clf.pkl')
    # hossain_n_estimators_600_learning_rate_0p1_max_depth_5_gbt_clf = load_model('./saved/models/hossain_n_estimators_600_learning_rate_0p1_max_depth_5_gbt_clf.pkl')
    # hossain_n_estimators_600_learning_rate_0p01_max_depth_5_gbt_clf = load_model('./saved/models/hossain_n_estimators_600_learning_rate_0p01_max_depth_5_gbt_clf.pkl')
    # hossain_n_estimators_600_learning_rate_0p1_max_depth_10_gbt_clf = load_model('./saved/models/hossain_n_estimators_600_learning_rate_0p1_max_depth_10_gbt_clf.pkl')
    # hossain_n_estimators_600_learning_rate_0p01_max_depth_10_gbt_clf = load_model('./saved/models/hossain_n_estimators_600_learning_rate_0p01_max_depth_10_gbt_clf.pkl')

    # taylor_n_estimators_600_max_depth_50_class_weight_None_rf = load_model('./saved/models/taylor_n_estimators_600_max_depth_50_class_weight_None_rf_clf.pkl')
    # taylor_n_estimators_400_max_depth_50_class_weight_None_rf = load_model('./saved/models/taylor_n_estimators_400_max_depth_50_class_weight_None_rf_clf.pkl')
    # taylor_n_estimators_200_max_depth_50_class_weight_None_rf = load_model('./saved/models/taylor_n_estimators_200_max_depth_50_class_weight_None_rf_clf.pkl')
    # taylor_n_estimators_600_max_depth_30_class_weight_None_rf = load_model('./saved/models/taylor_n_estimators_600_max_depth_30_class_weight_None_rf_clf.pkl')
    # taylor_n_estimators_400_max_depth_30_class_weight_None_rf = load_model('./saved/models/taylor_n_estimators_400_max_depth_30_class_weight_None_rf_clf.pkl')
    # taylor_n_estimators_200_max_depth_30_class_weight_None_rf = load_model('./saved/models/taylor_n_estimators_200_max_depth_30_class_weight_None_rf_clf.pkl')
    # taylor_n_estimators_600_max_depth_10_class_weight_None_rf = load_model('./saved/models/taylor_n_estimators_600_max_depth_10_class_weight_None_rf_clf.pkl')
    # taylor_n_estimators_400_max_depth_10_class_weight_None_rf = load_model('./saved/models/taylor_n_estimators_400_max_depth_10_class_weight_None_rf_clf.pkl')
    # taylor_n_estimators_200_max_depth_10_class_weight_None_rf = load_model('./saved/models/taylor_n_estimators_200_max_depth_10_class_weight_None_rf_clf.pkl')

    # done hossain_C_1_gamma_1_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_1_gamma_1_probability_True_class_weight_None_svm_clf.pkl')
    # done hossain_C_1_gamma_0p1_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_1_gamma_0p1_probability_True_class_weight_None_svm_clf.pkl')
    # done hossain_C_1_gamma_0p01_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_1_gamma_0p01_probability_True_class_weight_None_svm_clf.pkl')
    # done hossain_C_1_gamma_0p001_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_1_gamma_0p001_probability_True_class_weight_None_svm_clf.pkl')
    # hossain_C_10_gamma_1_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_10_gamma_1_probability_True_class_weight_None_svm_clf.pkl')
    # hossain_C_10_gamma_0p1_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_10_gamma_0p1_probability_True_class_weight_None_svm_clf.pkl')
    # done hossain_C_10_gamma_0p01_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_10_gamma_0p01_probability_True_class_weight_None_svm_clf.pkl')
    # done hossain_C_10_gamma_0p001_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_10_gamma_0p001_probability_True_class_weight_None_svm_clf.pkl')
    # hossain_C_100_gamma_1_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_100_gamma_1_probability_True_class_weight_None_svm_clf.pkl')
    # hossain_C_100_gamma_0p1_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_100_gamma_0p1_probability_True_class_weight_None_svm_clf.pkl')
    # done hossain_C_100_gamma_0p01_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_100_gamma_0p01_probability_True_class_weight_None_svm_clf.pkl')
    # done hossain_C_100_gamma_0p001_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_100_gamma_0p001_probability_True_class_weight_None_svm_clf.pkl')
    # hossain_C_1000_gamma_1_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_1000_gamma_1_probability_True_class_weight_None_svm_clf.pkl')
    # hossain_C_1000_gamma_0p1_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_1000_gamma_0p1_probability_True_class_weight_None_svm_clf.pkl')
    # hossain_C_1000_gamma_0p01_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_1000_gamma_0p01_probability_True_class_weight_None_svm_clf.pkl')
    hossain_C_1000_gamma_0p001_probability_True_class_weight_None_svm = load_model('./saved/models/hossain_C_1000_gamma_0p001_probability_True_class_weight_None_svm_clf.pkl')

    # done taylor_C_1_gamma_1_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_1_gamma_1_probability_True_class_weight_None_svm_clf.pkl')
    # done taylor_C_1_gamma_0p1_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_1_gamma_0p1_probability_True_class_weight_None_svm_clf.pkl')
    # done taylor_C_1_gamma_0p01_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_1_gamma_0p01_probability_True_class_weight_None_svm_clf.pkl')
    # done taylor_C_1_gamma_0p001_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_1_gamma_0p001_probability_True_class_weight_None_svm_clf.pkl')
    # taylor_C_10_gamma_1_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_10_gamma_1_probability_True_class_weight_None_svm_clf.pkl')
    # done taylor_C_10_gamma_0p1_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_10_gamma_0p1_probability_True_class_weight_None_svm_clf.pkl')
    # taylor_C_10_gamma_0p01_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_10_gamma_0p01_probability_True_class_weight_None_svm_clf.pkl')
    # done taylor_C_10_gamma_0p001_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_10_gamma_0p001_probability_True_class_weight_None_svm_clf.pkl')
    # taylor_C_100_gamma_1_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_100_gamma_1_probability_True_class_weight_None_svm_clf.pkl')
    # done taylor_C_100_gamma_0p1_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_100_gamma_0p1_probability_True_class_weight_None_svm_clf.pkl')
    # done taylor_C_100_gamma_0p01_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_100_gamma_0p01_probability_True_class_weight_None_svm_clf.pkl')
    # done taylor_C_100_gamma_0p001_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_100_gamma_0p001_probability_True_class_weight_None_svm_clf.pkl')
    # taylor_C_1000_gamma_1_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_1000_gamma_1_probability_True_class_weight_None_svm_clf.pkl')
    taylor_C_1000_gamma_0p1_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_1000_gamma_0p1_probability_True_class_weight_None_svm_clf.pkl')
    # taylor_C_1000_gamma_0p01_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_1000_gamma_0p01_probability_True_class_weight_None_svm_clf.pkl')
    # done taylor_C_1000_gamma_0p001_probability_True_class_weight_None_svm = load_model('./saved/models/taylor_C_1000_gamma_0p001_probability_True_class_weight_None_svm_clf.pkl')

    # hossain_C_0p01_class_weight_None_lr = load_model('./saved/models/hossain_C_0p01_class_weight_None_lr_clf.pkl')
    # hossain_C_0p1_class_weight_None_lr = load_model('./saved/models/hossain_C_0p1_class_weight_None_lr_clf.pkl')
    # hossain_C_1_class_weight_None_lr = load_model('./saved/models/hossain_C_1_class_weight_None_lr_clf.pkl')
    # hossain_C_10_class_weight_None_lr = load_model('./saved/models/hossain_C_10_class_weight_None_lr_clf.pkl')
    # hossain_C_100_class_weight_None_lr = load_model('./saved/models/hossain_C_100_class_weight_None_lr_clf.pkl')

    # taylor_C_0p01_class_weight_None_lr = load_model('./saved/models/taylor_C_0p01_class_weight_None_lr_clf.pkl')
    # taylor_C_0p1_class_weight_None_lr = load_model('./saved/models/taylor_C_0p1_class_weight_None_lr_clf.pkl')
    # taylor_C_1_class_weight_None_lr = load_model('./saved/models/taylor_C_1_class_weight_None_lr_clf.pkl')
    # taylor_C_10_class_weight_None_lr = load_model('./saved/models/taylor_C_10_class_weight_None_lr_clf.pkl')
    # taylor_C_100_class_weight_None_lr = load_model('./saved/models/taylor_C_100_class_weight_None_lr_clf.pkl')

    # models["hossain-n_estimators_200_learning_rate_0p1_max_depth_3_gbt"]['model'] = hossain_n_estimators_200_learning_rate_0p1_max_depth_3_gbt_clf
    # models["hossain-n_estimators_200_learning_rate_0p01_max_depth_3_gbt"]['model'] = hossain_n_estimators_200_learning_rate_0p01_max_depth_3_gbt_clf
    # models["hossain-n_estimators_200_learning_rate_0p1_max_depth_5_gbt"]['model'] = hossain_n_estimators_200_learning_rate_0p1_max_depth_5_gbt_clf
    # models["hossain-n_estimators_200_learning_rate_0p01_max_depth_5_gbt"]['model'] = hossain_n_estimators_200_learning_rate_0p01_max_depth_5_gbt_clf
    # models["hossain-n_estimators_200_learning_rate_0p1_max_depth_10_gbt"]['model'] = hossain_n_estimators_200_learning_rate_0p1_max_depth_10_gbt_clf
    # models["hossain-n_estimators_200_learning_rate_0p01_max_depth_10_gbt"]['model'] = hossain_n_estimators_200_learning_rate_0p01_max_depth_10_gbt_clf
    # models["hossain-n_estimators_400_learning_rate_0p1_max_depth_3_gbt"]['model'] = hossain_n_estimators_400_learning_rate_0p1_max_depth_3_gbt_clf
    # models["hossain-n_estimators_400_learning_rate_0p01_max_depth_3_gbt"]['model'] = hossain_n_estimators_400_learning_rate_0p01_max_depth_3_gbt_clf
    # models["hossain-n_estimators_400_learning_rate_0p1_max_depth_5_gbt"]['model'] = hossain_n_estimators_400_learning_rate_0p1_max_depth_5_gbt_clf
    # models["hossain-n_estimators_400_learning_rate_0p01_max_depth_5_gbt"]['model'] = hossain_n_estimators_400_learning_rate_0p01_max_depth_5_gbt_clf
    # models["hossain-n_estimators_400_learning_rate_0p1_max_depth_10_gbt"]['model'] = hossain_n_estimators_400_learning_rate_0p1_max_depth_10_gbt_clf
    # models["hossain-n_estimators_400_learning_rate_0p01_max_depth_10_gbt"]['model'] = hossain_n_estimators_400_learning_rate_0p01_max_depth_10_gbt_clf
    # models["hossain-n_estimators_600_learning_rate_0p1_max_depth_3_gbt"]['model'] = hossain_n_estimators_600_learning_rate_0p1_max_depth_3_gbt_clf
    # models["hossain-n_estimators_600_learning_rate_0p01_max_depth_3_gbt"]['model'] = hossain_n_estimators_600_learning_rate_0p01_max_depth_3_gbt_clf
    # models["hossain-n_estimators_600_learning_rate_0p1_max_depth_5_gbt"]['model'] = hossain_n_estimators_600_learning_rate_0p1_max_depth_5_gbt_clf
    # models["hossain-n_estimators_600_learning_rate_0p01_max_depth_5_gbt"]['model'] = hossain_n_estimators_600_learning_rate_0p01_max_depth_5_gbt_clf
    # models["hossain-n_estimators_600_learning_rate_0p1_max_depth_10_gbt"]['model'] = hossain_n_estimators_600_learning_rate_0p1_max_depth_10_gbt_clf
    # models["hossain-n_estimators_600_learning_rate_0p01_max_depth_10_gbt"]['model'] = hossain_n_estimators_600_learning_rate_0p01_max_depth_10_gbt_clf

    # models["taylor-n_estimators_600_max_depth_50_class_weight_None_rf"]['model'] = taylor_n_estimators_600_max_depth_50_class_weight_None_rf
    # models["taylor-n_estimators_400_max_depth_50_class_weight_None_rf"]['model'] = taylor_n_estimators_400_max_depth_50_class_weight_None_rf
    # models["taylor-n_estimators_200_max_depth_50_class_weight_None_rf"]['model'] = taylor_n_estimators_200_max_depth_50_class_weight_None_rf
    # models["taylor-n_estimators_600_max_depth_30_class_weight_None_rf"]['model'] = taylor_n_estimators_600_max_depth_30_class_weight_None_rf
    # models["taylor-n_estimators_400_max_depth_30_class_weight_None_rf"]['model'] = taylor_n_estimators_400_max_depth_30_class_weight_None_rf
    # models["taylor-n_estimators_200_max_depth_30_class_weight_None_rf"]['model'] = taylor_n_estimators_200_max_depth_30_class_weight_None_rf
    # models["taylor-n_estimators_600_max_depth_10_class_weight_None_rf"]['model'] = taylor_n_estimators_600_max_depth_10_class_weight_None_rf
    # models["taylor-n_estimators_400_max_depth_10_class_weight_None_rf"]['model'] = taylor_n_estimators_400_max_depth_10_class_weight_None_rf
    # models["taylor-n_estimators_200_max_depth_10_class_weight_None_rf"]['model'] = taylor_n_estimators_200_max_depth_10_class_weight_None_rf

    # done models["hossain-C_1_gamma_1_probability_True_class_weight_None_svm"]['model'] = hossain_C_1_gamma_1_probability_True_class_weight_None_svm
    # done models["hossain-C_1_gamma_0p1_probability_True_class_weight_None_svm"]['model'] = hossain_C_1_gamma_0p1_probability_True_class_weight_None_svm
    # done models["hossain-C_1_gamma_0p01_probability_True_class_weight_None_svm"]['model'] = hossain_C_1_gamma_0p01_probability_True_class_weight_None_svm
    # done models["hossain-C_1_gamma_0p001_probability_True_class_weight_None_svm"]['model'] = hossain_C_1_gamma_0p001_probability_True_class_weight_None_svm
    # models["hossain-C_10_gamma_1_probability_True_class_weight_None_svm"]['model'] = hossain_C_10_gamma_1_probability_True_class_weight_None_svm
    # models["hossain-C_10_gamma_0p1_probability_True_class_weight_None_svm"]['model'] = hossain_C_10_gamma_0p1_probability_True_class_weight_None_svm
    # done models["hossain-C_10_gamma_0p01_probability_True_class_weight_None_svm"]['model'] = hossain_C_10_gamma_0p01_probability_True_class_weight_None_svm
    # done models["hossain-C_10_gamma_0p001_probability_True_class_weight_None_svm"]['model'] = hossain_C_10_gamma_0p001_probability_True_class_weight_None_svm
    # models["hossain-C_100_gamma_1_probability_True_class_weight_None_svm"]['model'] = hossain_C_100_gamma_1_probability_True_class_weight_None_svm
    # models["hossain-C_100_gamma_0p1_probability_True_class_weight_None_svm"]['model'] = hossain_C_100_gamma_0p1_probability_True_class_weight_None_svm
    # done models["hossain-C_100_gamma_0p01_probability_True_class_weight_None_svm"]['model'] = hossain_C_100_gamma_0p01_probability_True_class_weight_None_svm
    # done models["hossain-C_100_gamma_0p001_probability_True_class_weight_None_svm"]['model'] = hossain_C_100_gamma_0p001_probability_True_class_weight_None_svm
    # models["hossain-C_1000_gamma_1_probability_True_class_weight_None_svm"]['model'] = hossain_C_1000_gamma_1_probability_True_class_weight_None_svm
    # models["hossain-C_1000_gamma_0p1_probability_True_class_weight_None_svm"]['model'] = hossain_C_1000_gamma_0p1_probability_True_class_weight_None_svm
    # models["hossain-C_1000_gamma_0p01_probability_True_class_weight_None_svm"]['model'] = hossain_C_1000_gamma_0p01_probability_True_class_weight_None_svm
    models["hossain-C_1000_gamma_0p001_probability_True_class_weight_None_svm"]['model'] = hossain_C_1000_gamma_0p001_probability_True_class_weight_None_svm

    # done models["taylor-C_1_gamma_1_probability_True_class_weight_None_svm"]['model'] = taylor_C_1_gamma_1_probability_True_class_weight_None_svm
    # done models["taylor-C_1_gamma_0p1_probability_True_class_weight_None_svm"]['model'] = taylor_C_1_gamma_0p1_probability_True_class_weight_None_svm
    # done models["taylor-C_1_gamma_0p01_probability_True_class_weight_None_svm"]['model'] = taylor_C_1_gamma_0p01_probability_True_class_weight_None_svm
    # done models["taylor-C_1_gamma_0p001_probability_True_class_weight_None_svm"]['model'] = taylor_C_1_gamma_0p001_probability_True_class_weight_None_svm
    # models["taylor-C_10_gamma_1_probability_True_class_weight_None_svm"]['model'] = taylor_C_10_gamma_1_probability_True_class_weight_None_svm
    # done models["taylor-C_10_gamma_0p1_probability_True_class_weight_None_svm"]['model'] = taylor_C_10_gamma_0p1_probability_True_class_weight_None_svm
    # models["taylor-C_10_gamma_0p01_probability_True_class_weight_None_svm"]['model'] = taylor_C_10_gamma_0p01_probability_True_class_weight_None_svm
    # done models["taylor-C_10_gamma_0p001_probability_True_class_weight_None_svm"]['model'] = taylor_C_10_gamma_0p001_probability_True_class_weight_None_svm
    # models["taylor-C_100_gamma_1_probability_True_class_weight_None_svm"]['model'] = taylor_C_100_gamma_1_probability_True_class_weight_None_svm
    # done models["taylor-C_100_gamma_0p1_probability_True_class_weight_None_svm"]['model'] = taylor_C_100_gamma_0p1_probability_True_class_weight_None_svm
    # done models["taylor-C_100_gamma_0p01_probability_True_class_weight_None_svm"]['model'] = taylor_C_100_gamma_0p01_probability_True_class_weight_None_svm
    # done models["taylor-C_100_gamma_0p001_probability_True_class_weight_None_svm"]['model'] = taylor_C_100_gamma_0p001_probability_True_class_weight_None_svm
    # models["taylor-C_1000_gamma_1_probability_True_class_weight_None_svm"]['model'] = taylor_C_1000_gamma_1_probability_True_class_weight_None_svm
    models["taylor-C_1000_gamma_0p1_probability_True_class_weight_None_svm"]['model'] = taylor_C_1000_gamma_0p1_probability_True_class_weight_None_svm
    # models["taylor-C_1000_gamma_0p01_probability_True_class_weight_None_svm"]['model'] = taylor_C_1000_gamma_0p01_probability_True_class_weight_None_svm
    # done models["taylor-C_1000_gamma_0p001_probability_True_class_weight_None_svm"]['model'] = taylor_C_1000_gamma_0p001_probability_True_class_weight_None_svm

    # models["hossain-C_0p01_class_weight_None_lr"]['model'] = hossain_C_0p01_class_weight_None_lr
    # models["hossain-C_0p1_class_weight_None_lr"]['model'] = hossain_C_0p1_class_weight_None_lr
    # models["hossain-C_1_class_weight_None_lr"]['model'] = hossain_C_1_class_weight_None_lr
    # models["hossain-C_10_class_weight_None_lr"]['model'] = hossain_C_10_class_weight_None_lr
    # models["hossain-C_100_class_weight_None_lr"]['model'] = hossain_C_100_class_weight_None_lr

    # models["taylor-C_0p01_class_weight_None_lr"]['model'] = taylor_C_0p01_class_weight_None_lr
    # models["taylor-C_0p1_class_weight_None_lr"]['model'] = taylor_C_0p1_class_weight_None_lr
    # models["taylor-C_1_class_weight_None_lr"]['model'] = taylor_C_1_class_weight_None_lr
    # models["taylor-C_10_class_weight_None_lr"]['model'] = taylor_C_10_class_weight_None_lr
    # models["taylor-C_100_class_weight_None_lr"]['model'] = taylor_C_100_class_weight_None_lr

    print('models loaded.')

In [9]:
load_miscs()
load_preprocessors()
load_models()

loading miscellaneous...
miscellaneous loaded.
loading preprocessors...
preprocessors loaded.
loading models...
models loaded.


In [10]:
models

{'hossain-C_1000_gamma_0p001_probability_True_class_weight_None_svm': {'selected_feats': ['raw_128hz_max',
   'raw_128hz_min',
   'raw_128hz_amp',
   'raw_128hz_median',
   'raw_128hz_std',
   'raw_128hz_range',
   'raw_128hz_shannon_entropy',
   'raw_128hz_1d_max',
   'raw_128hz_1d_min',
   'raw_128hz_1d_amp',
   'raw_128hz_1d_std',
   'raw_128hz_1d_range',
   'raw_128hz_1d_shannon_entropy',
   'filt_128hz_max',
   'filt_128hz_min',
   'filt_128hz_amp',
   'filt_128hz_median',
   'filt_128hz_std',
   'filt_128hz_range',
   'filt_128hz_1d_max',
   'filt_128hz_1d_min',
   'filt_128hz_1d_amp',
   'filt_128hz_1d_std',
   'filt_128hz_1d_range',
   'ar_coeff_1_128hz',
   'ar_coeff_2_128hz',
   'ar_err_var_128hz',
   'vfcdm_4/8_128hz_mean',
   'first_16thofa_sec_mean',
   'second_16thofa_sec_mean',
   'third_16thofa_sec_mean',
   'first_16thofa_sec_std',
   'second_16thofa_sec_std',
   'third_16thofa_sec_std',
   'third_16thofa_sec_median',
   'third_16thofa_sec_range',
   'first_32thofa_sec

In [11]:
model_names = [
    # done "hossain-C_1_gamma_1_probability_True_class_weight_None_svm",
    # done "hossain-C_1_gamma_0p1_probability_True_class_weight_None_svm",
    # done "hossain-C_1_gamma_0p01_probability_True_class_weight_None_svm",
    # done "hossain-C_1_gamma_0p001_probability_True_class_weight_None_svm",
    # "hossain-C_10_gamma_1_probability_True_class_weight_None_svm",
    # "hossain-C_10_gamma_0p1_probability_True_class_weight_None_svm",
    # done "hossain-C_10_gamma_0p01_probability_True_class_weight_None_svm",
    # done "hossain-C_10_gamma_0p001_probability_True_class_weight_None_svm",
    # "hossain-C_100_gamma_1_probability_True_class_weight_None_svm",
    # "hossain-C_100_gamma_0p1_probability_True_class_weight_None_svm",
    # done "hossain-C_100_gamma_0p01_probability_True_class_weight_None_svm",
    # done "hossain-C_100_gamma_0p001_probability_True_class_weight_None_svm",
    # "hossain-C_1000_gamma_1_probability_True_class_weight_None_svm",
    # "hossain-C_1000_gamma_0p1_probability_True_class_weight_None_svm",
    # "hossain-C_1000_gamma_0p01_probability_True_class_weight_None_svm",
    "hossain-C_1000_gamma_0p001_probability_True_class_weight_None_svm",

    # done "taylor-C_1_gamma_1_probability_True_class_weight_None_svm",
    # done "taylor-C_1_gamma_0p1_probability_True_class_weight_None_svm",
    # done "taylor-C_1_gamma_0p01_probability_True_class_weight_None_svm",
    # done "taylor-C_1_gamma_0p001_probability_True_class_weight_None_svm",
    # "taylor-C_10_gamma_1_probability_True_class_weight_None_svm",
    # "taylor-C_10_gamma_0p1_probability_True_class_weight_None_svm",
    # "taylor-C_10_gamma_0p01_probability_True_class_weight_None_svm",
    # done "taylor-C_10_gamma_0p001_probability_True_class_weight_None_svm",
    # "taylor-C_100_gamma_1_probability_True_class_weight_None_svm",
    # done "taylor-C_100_gamma_0p1_probability_True_class_weight_None_svm",
    # done "taylor-C_100_gamma_0p01_probability_True_class_weight_None_svm",
    # done "taylor-C_100_gamma_0p001_probability_True_class_weight_None_svm",
    # "taylor-C_1000_gamma_1_probability_True_class_weight_None_svm",
    "taylor-C_1000_gamma_0p1_probability_True_class_weight_None_svm",
    # "taylor-C_1000_gamma_0p01_probability_True_class_weight_None_svm",
    # done "taylor-C_1000_gamma_0p001_probability_True_class_weight_None_svm",

    # "hossain-C_0p01_class_weight_None_lr",
    # "hossain-C_0p1_class_weight_None_lr",
    # "hossain-C_1_class_weight_None_lr",
    # "hossain-C_10_class_weight_None_lr",
    # "hossain-C_100_class_weight_None_lr",
    
    # "taylor-C_0p01_class_weight_None_lr",
    # "taylor-C_0p1_class_weight_None_lr",
    # "taylor-C_1_class_weight_None_lr",
    # "taylor-C_10_class_weight_None_lr",
    # "taylor-C_100_class_weight_None_lr",
]
data_splits = ["test"]

In [12]:
for model_name in model_names:
    selector_config, estimator_name = model_name.split('-', 1)
    
    for data_split in data_splits:
        loader_args = {
            'feat_config': selector_config, 
            'data_split': data_split,
            'exc_lof': False
        } if selector_config == 'cueva_second_phase' else {
            'feat_config': selector_config, 
            'data_split': data_split,
            'exc_lof': False
        }

        subjects_features, subjects_labels, subjects_names, subject_to_id = concur_load_data(**loader_args)
        print(f'selector config: {selector_config}')
        print(f'estimator name: {estimator_name}')
        # print(f'subjects features shape: {subjects_features.shape}')
        # print(f'subjects labels shape: {subjects_labels.shape}\n')
        
        # loop through each generated features dataframes from test subjects signals and feed repeatedly to a trained ml models
        for index, subject_name in enumerate(subjects_names):
            # print(f'subject features columns: {subjects_features[subjects_features['subject_id'] == index].columns}')
            print(f'subject: {subject_name}')
            # once features are extracted features selected during
            # tuning will be used in testing as done also during training

            selected_feats = models[model_name]['selected_feats']

            if loader_args.get('exc_lof') == None or loader_args.get('exc_lof') == False:
                subject_features = subjects_features.loc[subjects_features['subject_id'] == index, selected_feats]
                subject_labels = subjects_labels[subjects_labels['subject_id'] == index].drop(columns=['subject_id'])
                print(f'subject features shape: {subject_features.shape}')
                print(f'subject labels shape: {subject_labels.shape}\n')

            # this will only fire if exc_lof is not None or is false
            else:
                # if user excludes lower order features, higher order features will only be loaded
                subject_features = subjects_features[subjects_features['subject_id'] == index].drop(columns=['subject_id'])
                subject_labels = subjects_labels[subjects_labels['subject_id'] == index].drop(columns=['subject_id'])
                
                print(f'subject features shape: {subject_features.shape}')
                print(f'subject labels shape: {subject_labels.shape}\n')

            # convert features and labels into numpy matrices
            X = subject_features.to_numpy()
            Y = subject_labels.to_numpy().ravel()

            # if hossain is the researcher chosen the scaler used during training
            # will be used to scale the test subject features
            if selector_config == "hossain" or selector_config == "cueva_second_phase":    
                scaler = models[model_name]['scaler']
                X = scaler.transform(X)

            model = models[model_name]['model']
            Y_pred = model.predict(X)
            Y_pred_prob = model.predict_proba(X)
            print(f"predicted Y: {Y_pred}")
            print(f"unique values and counts: {np.unique(Y_pred, return_counts=True)}")
            print(f"true Y: {Y}")
            print(f"unique values and counts: {np.unique(Y, return_counts=True)}")

            # compute performance metric values for test subject
            acc = accuracy_score(y_true=Y, y_pred=Y_pred)
            prec = precision_score(y_true=Y, y_pred=Y_pred, average="weighted")
            rec = recall_score(y_true=Y, y_pred=Y_pred, average="weighted")
            f1 = f1_score(y_true=Y, y_pred=Y_pred, average="weighted")
            roc_auc = roc_auc_score(y_true=Y, y_score=Y_pred_prob[:, 1], average="weighted", multi_class="ovo")
            conf_matrix = confusion_matrix(Y, Y_pred).tolist()
            true_neg = conf_matrix[0][0]
            false_pos = conf_matrix[0][1]
            false_neg = conf_matrix[1][0]
            true_pos = conf_matrix[1][1]
            tpr = true_pos / (true_pos + false_neg)
            tnr = true_neg / (true_neg + false_pos)
            fpr = false_pos / (false_pos + true_neg)
            fnr = false_neg / (false_neg + true_pos)

            print(f"{data_split} acc: {acc} \
                \n{data_split} prec: {prec} \
                \n{data_split} rec: {rec} \
                \n{data_split} f1: {f1} \
                \n{data_split} roc_auc: {roc_auc} \
                \n{data_split} conf_matrix: {conf_matrix} \
                \n{data_split} tpr: {tpr} \
                \n{data_split} tnr: {tnr} \
                \n{data_split} fpr: {fpr} \
                \n{data_split} fnr: {fnr}")
            
            results = models[model_name].get(f'{data_split}_results', [])
            results.append(
                (subject_name, {
                    f'{data_split}_acc': acc,
                    f'{data_split}_prec': prec, 
                    f'{data_split}_rec': rec,
                    f'{data_split}_f1': f1,
                    f'{data_split}_roc_auc': roc_auc,
                    f'{data_split}_conf_matrix': conf_matrix,
                    f'{data_split}_tpr': tpr,
                    f'{data_split}_tnr': tnr,
                    f'{data_split}_fpr': fpr,
                    f'{data_split}_fnr': fnr,
                })
            )
            models[f'{model_name}'][f'{data_split}_results'] = results

subjects features, labels, names and subject to id lookup loaded
selector config: hossain
estimator name: C_1000_gamma_0p001_probability_True_class_weight_None_svm
subject: inefoh_expert1
subject features shape: (13377, 40)
subject labels shape: (13377, 1)

predicted Y: [0. 0. 0. ... 0. 0. 0.]
unique values and counts: (array([0., 1.]), array([13154,   223], dtype=int64))
true Y: [0. 0. 0. ... 0. 0. 0.]
unique values and counts: (array([0., 1.]), array([12517,   860], dtype=int64))
test acc: 0.946251027883681                 
test prec: 0.9399503971485317                 
test rec: 0.946251027883681                 
test f1: 0.9311108296642004                 
test roc_auc: 0.8073944087204192                 
test conf_matrix: [[12476, 41], [678, 182]]                 
test tpr: 0.2116279069767442                 
test tnr: 0.9967244547415515                 
test fpr: 0.00327554525844851                 
test fnr: 0.7883720930232558
subject: pqbqpr_expert2
subject features shape: (153

In [13]:
copy = models.copy()

In [14]:
for key, value in copy.items():
    if copy[key].get('model') is not None:
        del copy[key]['model']

    if copy[key].get('scaler') is not None:
        del copy[key]['scaler']

In [15]:
copy

{'hossain-C_1000_gamma_0p001_probability_True_class_weight_None_svm': {'selected_feats': ['raw_128hz_max',
   'raw_128hz_min',
   'raw_128hz_amp',
   'raw_128hz_median',
   'raw_128hz_std',
   'raw_128hz_range',
   'raw_128hz_shannon_entropy',
   'raw_128hz_1d_max',
   'raw_128hz_1d_min',
   'raw_128hz_1d_amp',
   'raw_128hz_1d_std',
   'raw_128hz_1d_range',
   'raw_128hz_1d_shannon_entropy',
   'filt_128hz_max',
   'filt_128hz_min',
   'filt_128hz_amp',
   'filt_128hz_median',
   'filt_128hz_std',
   'filt_128hz_range',
   'filt_128hz_1d_max',
   'filt_128hz_1d_min',
   'filt_128hz_1d_amp',
   'filt_128hz_1d_std',
   'filt_128hz_1d_range',
   'ar_coeff_1_128hz',
   'ar_coeff_2_128hz',
   'ar_err_var_128hz',
   'vfcdm_4/8_128hz_mean',
   'first_16thofa_sec_mean',
   'second_16thofa_sec_mean',
   'third_16thofa_sec_mean',
   'first_16thofa_sec_std',
   'second_16thofa_sec_std',
   'third_16thofa_sec_std',
   'third_16thofa_sec_median',
   'third_16thofa_sec_range',
   'first_32thofa_sec

In [16]:
# done all_models_results["hossain-C_1_gamma_1_probability_True_class_weight_None_svm"] = copy["hossain-C_1_gamma_1_probability_True_class_weight_None_svm"]
# done all_models_results["hossain-C_1_gamma_0p1_probability_True_class_weight_None_svm"] = copy["hossain-C_1_gamma_0p1_probability_True_class_weight_None_svm"]
# done all_models_results["hossain-C_1_gamma_0p01_probability_True_class_weight_None_svm"] = copy["hossain-C_1_gamma_0p01_probability_True_class_weight_None_svm"]
# done all_models_results["hossain-C_1_gamma_0p001_probability_True_class_weight_None_svm"] = copy["hossain-C_1_gamma_0p001_probability_True_class_weight_None_svm"]
# all_models_results["hossain-C_10_gamma_1_probability_True_class_weight_None_svm"] = copy["hossain-C_10_gamma_1_probability_True_class_weight_None_svm"]
# all_models_results["hossain-C_10_gamma_0p1_probability_True_class_weight_None_svm"] = copy["hossain-C_10_gamma_0p1_probability_True_class_weight_None_svm"]
# done all_models_results["hossain-C_10_gamma_0p01_probability_True_class_weight_None_svm"] = copy["hossain-C_10_gamma_0p01_probability_True_class_weight_None_svm"]
# done all_models_results["hossain-C_10_gamma_0p001_probability_True_class_weight_None_svm"] = copy["hossain-C_10_gamma_0p001_probability_True_class_weight_None_svm"]
# all_models_results["hossain-C_100_gamma_1_probability_True_class_weight_None_svm"] = copy["hossain-C_100_gamma_1_probability_True_class_weight_None_svm"]
# all_models_results["hossain-C_100_gamma_0p1_probability_True_class_weight_None_svm"] = copy["hossain-C_100_gamma_0p1_probability_True_class_weight_None_svm"]
# done all_models_results["hossain-C_100_gamma_0p01_probability_True_class_weight_None_svm"] = copy["hossain-C_100_gamma_0p01_probability_True_class_weight_None_svm"]
# done all_models_results["hossain-C_100_gamma_0p001_probability_True_class_weight_None_svm"] = copy["hossain-C_100_gamma_0p001_probability_True_class_weight_None_svm"]
# all_models_results["hossain-C_1000_gamma_1_probability_True_class_weight_None_svm"] = copy["hossain-C_1000_gamma_1_probability_True_class_weight_None_svm"]
# all_models_results["hossain-C_1000_gamma_0p1_probability_True_class_weight_None_svm"] = copy["hossain-C_1000_gamma_0p1_probability_True_class_weight_None_svm"]
# all_models_results["hossain-C_1000_gamma_0p01_probability_True_class_weight_None_svm"] = copy["hossain-C_1000_gamma_0p01_probability_True_class_weight_None_svm"]
all_models_results["hossain-C_1000_gamma_0p001_probability_True_class_weight_None_svm"] = copy["hossain-C_1000_gamma_0p001_probability_True_class_weight_None_svm"]

# done all_models_results["taylor-C_1_gamma_1_probability_True_class_weight_None_svm"] = copy["taylor-C_1_gamma_1_probability_True_class_weight_None_svm"]
# done all_models_results["taylor-C_1_gamma_0p1_probability_True_class_weight_None_svm"] = copy["taylor-C_1_gamma_0p1_probability_True_class_weight_None_svm"]
# done all_models_results["taylor-C_1_gamma_0p01_probability_True_class_weight_None_svm"] = copy["taylor-C_1_gamma_0p01_probability_True_class_weight_None_svm"]
# done all_models_results["taylor-C_1_gamma_0p001_probability_True_class_weight_None_svm"] = copy["taylor-C_1_gamma_0p001_probability_True_class_weight_None_svm"]
# all_models_results["taylor-C_10_gamma_1_probability_True_class_weight_None_svm"] = copy["taylor-C_10_gamma_1_probability_True_class_weight_None_svm"]
# all_models_results["taylor-C_10_gamma_0p1_probability_True_class_weight_None_svm"] = copy["taylor-C_10_gamma_0p1_probability_True_class_weight_None_svm"]
# all_models_results["taylor-C_10_gamma_0p01_probability_True_class_weight_None_svm"] = copy["taylor-C_10_gamma_0p01_probability_True_class_weight_None_svm"]
# done all_models_results["taylor-C_10_gamma_0p001_probability_True_class_weight_None_svm"] = copy["taylor-C_10_gamma_0p001_probability_True_class_weight_None_svm"]
# all_models_results["taylor-C_100_gamma_1_probability_True_class_weight_None_svm"] = copy["taylor-C_100_gamma_1_probability_True_class_weight_None_svm"]
# done all_models_results["taylor-C_100_gamma_0p1_probability_True_class_weight_None_svm"] = copy["taylor-C_100_gamma_0p1_probability_True_class_weight_None_svm"]
# done all_models_results["taylor-C_100_gamma_0p01_probability_True_class_weight_None_svm"] = copy["taylor-C_100_gamma_0p01_probability_True_class_weight_None_svm"]
# done all_models_results["taylor-C_100_gamma_0p001_probability_True_class_weight_None_svm"] = copy["taylor-C_100_gamma_0p001_probability_True_class_weight_None_svm"]
# all_models_results["taylor-C_1000_gamma_1_probability_True_class_weight_None_svm"] = copy["taylor-C_1000_gamma_1_probability_True_class_weight_None_svm"]
all_models_results["taylor-C_1000_gamma_0p1_probability_True_class_weight_None_svm"] = copy["taylor-C_1000_gamma_0p1_probability_True_class_weight_None_svm"]
# all_models_results["taylor-C_1000_gamma_0p01_probability_True_class_weight_None_svm"] = copy["taylor-C_1000_gamma_0p01_probability_True_class_weight_None_svm"]
# done all_models_results["taylor-C_1000_gamma_0p001_probability_True_class_weight_None_svm"] = copy["taylor-C_1000_gamma_0p001_probability_True_class_weight_None_svm"]

# all_models_results["hossain-C_0p01_class_weight_None_lr"] = copy["hossain-C_0p01_class_weight_None_lr"]
# all_models_results["hossain-C_0p1_class_weight_None_lr"] = copy["hossain-C_0p1_class_weight_None_lr"]
# all_models_results["hossain-C_1_class_weight_None_lr"] = copy["hossain-C_1_class_weight_None_lr"]
# all_models_results["hossain-C_10_class_weight_None_lr"] = copy["hossain-C_10_class_weight_None_lr"]
# all_models_results["hossain-C_100_class_weight_None_lr"] = copy["hossain-C_100_class_weight_None_lr"]

# all_models_results["taylor-C_0p01_class_weight_None_lr"] = copy["taylor-C_0p01_class_weight_None_lr"]
# all_models_results["taylor-C_0p1_class_weight_None_lr"] = copy["taylor-C_0p1_class_weight_None_lr"]
# all_models_results["taylor-C_1_class_weight_None_lr"] = copy["taylor-C_1_class_weight_None_lr"]
# all_models_results["taylor-C_10_class_weight_None_lr"] = copy["taylor-C_10_class_weight_None_lr"]
# all_models_results["taylor-C_100_class_weight_None_lr"] = copy["taylor-C_100_class_weight_None_lr"]

In [17]:
all_models_results

{'cueva_second_phase-1-5-weighted-svm': {'selected_feats': ['HOF_2',
   'HOF_5',
   'HOF_7',
   'HOF_9',
   'HOF_10',
   'HOF_12',
   'HOF_13',
   'HOF_17',
   'HOF_18',
   'HOF_19',
   'HOF_20',
   'HOF_21',
   'HOF_22',
   'HOF_26',
   'HOF_32',
   'raw_128hz_std',
   'raw_128hz_range',
   'raw_128hz_shannon_entropy',
   'raw_128hz_1d_shannon_entropy',
   'filt_128hz_min',
   'filt_128hz_1d_range',
   'filt_128hz_1d_max_abs',
   'ar_coeff_1_128hz',
   'ar_coeff_2_128hz',
   'raw_128hz_skewness',
   'raw_128hz_kurt',
   'third_16thofa_sec_max',
   'first_16thofa_sec_mean',
   'second_16thofa_sec_mean',
   'third_16thofa_sec_std',
   'first_16thofa_sec_median',
   'second_16thofa_sec_median',
   'third_16thofa_sec_range',
   'second_32thofa_sec_max',
   'first_32thofa_sec_mean',
   'second_32thofa_sec_mean',
   'first_32thofa_sec_std',
   'second_32thofa_sec_std',
   'first_32thofa_sec_median',
   'second_32thofa_sec_median',
   'first_32thofa_sec_range',
   'raw_16hz_min',
   'raw_16h

In [18]:
save_meta_data('./results/all_models_results.json', all_models_results)